In [1]:
import os
import glob
import pandas as pd

In [2]:
data_patient = pd.read_csv('/content/drive/MyDrive/MSK-CHORD-new/data_clinical_patient.txt', sep='\t', skiprows = 4)
print(data_patient.columns)
col_pt = ["PATIENT_ID", "OS_MONTHS"]
df_pt = data_patient[col_pt]
df_pt.head()

Index(['PATIENT_ID', 'GENDER', 'RACE', 'ETHNICITY', 'CURRENT_AGE_DEID',
       'STAGE_HIGHEST_RECORDED', 'NUM_ICDO_DX', 'ADRENAL_GLANDS', 'BONE',
       'CNS_BRAIN', 'INTRA_ABDOMINAL', 'LIVER', 'LUNG', 'LYMPH_NODES', 'OTHER',
       'PLEURA', 'REPRODUCTIVE_ORGANS', 'SMOKING_PREDICTIONS_3_CLASSES',
       'GLEASON_FIRST_REPORTED', 'GLEASON_HIGHEST_REPORTED', 'HISTORY_OF_PDL1',
       'PRIOR_MED_TO_MSK', 'OS_MONTHS', 'OS_STATUS', 'HR', 'HER2'],
      dtype='object')


,PATIENT_ID,OS_MONTHS
0,P-0000012,118.454665
1,P-0000015,13.906834
2,P-0000036,115.462887
3,P-0000041,13.610944
4,P-0000066,76.635532


In [29]:
data_sample = pd.read_csv('/content/drive/MyDrive/MSK-CHORD-new/data_clinical_sample.txt', sep = '\t', skiprows = 4)
print(data_sample.columns)
col_samp = ["SAMPLE_ID", "PATIENT_ID", "CANCER_TYPE"]
df_samp = data_sample[col_samp]
df_samp.head()
cancer_samp_pt = df_samp["PATIENT_ID"].nunique()
print(cancer_samp_pt)
df_samp.head()

Index(['SAMPLE_ID', 'PATIENT_ID', 'GLEASON_SAMPLE_LEVEL', 'PDL1_POSITIVE',
       'CANCER_TYPE', 'SAMPLE_TYPE', 'SAMPLE_CLASS', 'METASTATIC_SITE',
       'PRIMARY_SITE', 'CANCER_TYPE_DETAILED', 'GENE_PANEL', 'SAMPLE_COVERAGE',
       'TUMOR_PURITY', 'ONCOTREE_CODE', 'MSI_COMMENT', 'MSI_SCORE', 'MSI_TYPE',
       'SOMATIC_STATUS', 'CLINICAL_GROUP', 'PATHOLOGICAL_GROUP',
       'CLINICAL_SUMMARY', 'ICD_O_HISTOLOGY_DESCRIPTION',
       'DIAGNOSIS_DESCRIPTION', 'TMB_NONSYNONYMOUS'],
      dtype='object')
24950


,SAMPLE_ID,PATIENT_ID,CANCER_TYPE
0,P-0000012-T03-IM3,P-0000012,Non-Small Cell Lung Cancer
1,P-0000012-T02-IM3,P-0000012,Breast Cancer
2,P-0000015-T01-IM3,P-0000015,Breast Cancer
3,P-0000036-T01-IM3,P-0000036,Non-Small Cell Lung Cancer
4,P-0000041-T01-IM3,P-0000041,Breast Cancer


In [27]:
data_mut = pd.read_csv('/content/drive/MyDrive/MSK-CHORD-new/data_mutations.txt', sep ='\t')
print(data_mut.columns)
#data_mut["PATIENT_ID"] = data_mut["Tumor_Sample_Barcode"].str.split("-").str[:2].str.join("-")
data_mut["SAMPLE_ID"] = data_mut["Tumor_Sample_Barcode"]
col_mut = ["SAMPLE_ID", "Patient_Identifier", "Hugo_Symbol", "Consequence", "Variant_Classification", "Variant_Type", "t_ref_count", "t_alt_count", "HGVSp_Short"]
df_mut = data_mut[col_mut]
df_pt_mut_unique = df_mut["Patient_Identifier"].nunique()
print(df_pt_mut_unique)
df_mut.head()

Index(['Hugo_Symbol', 'Entrez_Gene_Id', 'Center', 'NCBI_Build', 'Chromosome',
       'Start_Position', 'End_Position', 'Strand', 'Consequence',
       'Variant_Classification', 'Variant_Type', 'Reference_Allele',
       'Tumor_Seq_Allele1', 'Tumor_Seq_Allele2', 'dbSNP_RS',
       'dbSNP_Val_Status', 'Tumor_Sample_Barcode', 'Patient_Identifier',
       'Matched_Norm_Sample_Barcode', 'Match_Norm_Seq_Allele1',
       'Match_Norm_Seq_Allele2', 'Tumor_Validation_Allele1',
       'Tumor_Validation_Allele2', 'Match_Norm_Validation_Allele1',
       'Match_Norm_Validation_Allele2', 'Verification_Status',
       'Validation_Status', 'Mutation_Status', 'Sequencing_Phase',
       'Sequence_Source', 'Validation_Method', 'Score', 'BAM_File',
       'Sequencer', 't_ref_count', 't_alt_count', 'n_ref_count', 'n_alt_count',
       'HGVSc', 'HGVSp', 'HGVSp_Short', 'Transcript_ID', 'RefSeq',
       'Protein_position', 'Codons'],
      dtype='object')
23785


,SAMPLE_ID,Patient_Identifier,Hugo_Symbol,Consequence,Variant_Classification,Variant_Type,t_ref_count,t_alt_count,HGVSp_Short
0,P-0081657-T01-IM7,P-0081657,EGFR,inframe_deletion,In_Frame_Del,DEL,319,288,p.L747_P753delinsS
1,P-0081657-T01-IM7,P-0081657,PDGFRB,missense_variant,Missense_Mutation,SNP,370,85,p.H271R
2,P-0081657-T01-IM7,P-0081657,RBM10,frameshift_variant,Frame_Shift_Del,DEL,189,150,p.Y519Wfs*96
3,P-0083825-T01-IM7,P-0083825,TP53,missense_variant,Missense_Mutation,SNP,187,111,p.Y205C
4,P-0083825-T01-IM7,P-0083825,TP53,stop_gained,Nonsense_Mutation,SNP,222,125,p.E294*


In [45]:
temp = pd.merge(df_pt, df_samp, on = "PATIENT_ID")
n_pt_per_cancer = temp.groupby("CANCER_TYPE")["PATIENT_ID"].nunique()
merged_df = pd.merge(temp, df_mut, on = "SAMPLE_ID")
col_order = ["PATIENT_ID", "SAMPLE_ID", "CANCER_TYPE", "OS_MONTHS", "Hugo_Symbol", "Consequence", "Variant_Classification", "Variant_Type", "t_ref_count", "t_alt_count", "HGVSp_Short"]
merged_df = merged_df[col_order]
merged_df.head()

,PATIENT_ID,SAMPLE_ID,CANCER_TYPE,OS_MONTHS,Hugo_Symbol,Consequence,Variant_Classification,Variant_Type,t_ref_count,t_alt_count,HGVSp_Short
0,P-0000012,P-0000012-T03-IM3,Non-Small Cell Lung Cancer,118.454665,PIK3R3,missense_variant,Missense_Mutation,SNP,604,129,p.I298M
1,P-0000012,P-0000012-T03-IM3,Non-Small Cell Lung Cancer,118.454665,ATR,missense_variant,Missense_Mutation,SNP,398,84,p.R2431M
2,P-0000012,P-0000012-T03-IM3,Non-Small Cell Lung Cancer,118.454665,PDGFRA,missense_variant,Missense_Mutation,SNP,454,116,p.L465M
3,P-0000012,P-0000012-T03-IM3,Non-Small Cell Lung Cancer,118.454665,FBXW7,splice_acceptor_variant,Splice_Site,SNP,248,85,p.X413_splice
4,P-0000012,P-0000012-T03-IM3,Non-Small Cell Lung Cancer,118.454665,FBXW7,stop_gained,Nonsense_Mutation,SNP,440,130,p.G61*


In [34]:
no_mut = merged_df[merged_df["Hugo_Symbol"].isna()]
print("Patients with no mutation data:", no_mut["PATIENT_ID"].nunique())
no_mut.to_csv("/content/drive/MyDrive/MSK-CHORD-new/patients_with_no_mutations.csv", index=False)

Patients with no mutation data: 1173


In [38]:
#calculate VAF
merged_df["VAF"] = merged_df["t_alt_count"] / (merged_df["t_ref_count"] + merged_df["t_alt_count"])
merged_df.head()

,PATIENT_ID,SAMPLE_ID,CANCER_TYPE,OS_MONTHS,Hugo_Symbol,Consequence,Variant_Classification,Variant_Type,t_ref_count,t_alt_count,HGVSp_Short,VAF
0,P-0000012,P-0000012-T03-IM3,Non-Small Cell Lung Cancer,118.454665,PIK3R3,missense_variant,Missense_Mutation,SNP,604,129,p.I298M,0.175989
1,P-0000012,P-0000012-T03-IM3,Non-Small Cell Lung Cancer,118.454665,ATR,missense_variant,Missense_Mutation,SNP,398,84,p.R2431M,0.174274
2,P-0000012,P-0000012-T03-IM3,Non-Small Cell Lung Cancer,118.454665,PDGFRA,missense_variant,Missense_Mutation,SNP,454,116,p.L465M,0.203509
3,P-0000012,P-0000012-T03-IM3,Non-Small Cell Lung Cancer,118.454665,FBXW7,splice_acceptor_variant,Splice_Site,SNP,248,85,p.X413_splice,0.255255
4,P-0000012,P-0000012-T03-IM3,Non-Small Cell Lung Cancer,118.454665,FBXW7,stop_gained,Nonsense_Mutation,SNP,440,130,p.G61*,0.228070


In [40]:
#write out individual cancer files
cancer_types = merged_df["CANCER_TYPE"].unique()
for cancer in cancer_types:
  cancer_df = merged_df[merged_df["CANCER_TYPE"] == cancer]
  cancer_df.to_csv(f"/content/drive/MyDrive/MSK-CHORD-new/cancer_files/{cancer}.csv", index = False)

In [44]:
Alt_events = merged_df["Variant_Classification"].unique()
print(Alt_events)

['Missense_Mutation' 'Splice_Site' 'Nonsense_Mutation'
 'Translation_Start_Site' 'Frame_Shift_Del' 'Frame_Shift_Ins'
 'In_Frame_Del' 'In_Frame_Ins' "5'Flank" "3'Flank" 'Intron' "5'UTR"
 'Nonstop_Mutation' 'Splice_Region' 'Silent' 'IGR' "3'UTR" 'RNA'
 'nonsynonymous_SNV' 'frameshift_insertion']


In [46]:
# Define input/output directories
input_dir  = "/content/drive/MyDrive/MSK-CHORD-new/cancer_files"
output_dir = "/content/drive/MyDrive/MSK-CHORD-new/cancer_files_scored"
os.makedirs(output_dir, exist_ok=True)

# Loop through each cancer file
for filepath in glob.glob(os.path.join(input_dir, "*.csv")):
    # read
    file = pd.read_csv(filepath)

    # definiing the mutation events [that cause change in the protein]
    Mut_event = ["Missense_Mutation", "Nonsense_Mutation",
              "Frame_Shift_Del", "Frame_Shift_Ins",
              "In_Frame_Del", "In_Frame_Ins",
             "Splice_Site", "Nonstop_Mutation", "Translation_Start_Site"]

    # Extract number of patients in cancer type
    file['total_pateints'] = file['CANCER_TYPE'].map(n_pt_per_cancer)

    # Compute per-gene patient counts for mutation events only
    mask = file["Variant_Classification"].isin(Mut_event)

    n_patient_mut = (file[mask]
                     .drop_duplicates(subset = ["Hugo_Symbol", "PATIENT_ID"])
                     .groupby("Hugo_Symbol")["PATIENT_ID"]
                     .nunique())

    # Map those counts back and calculate Mut_freq
    file["n_patient_mut"] = file["Hugo_Symbol"].map(n_patient_mut).fillna(0).astype(int)
    file["Mut_freq"]    = file["n_patient_mut"] / file["total_pateints"]


    # Compute average OS_MONTHS per gene
    file["avg_OS_months"] = (file.groupby("Hugo_Symbol")["OS_MONTHS"]
                              .transform("mean"))

    # Compute genomic alterations per gene and the genomic alteration rate
    file["genomic_alt_cohort"] = file.groupby("Hugo_Symbol")["Variant_Classification"].transform("count")
    file["genomic_alt_rate_cohort"] = file["genomic_alt_cohort"]/ file["total_pateints"]

    # gene score
    file["Gene_Score"] = file["Mut_freq"] * file["avg_OS_months"]

    # drop the patient column, then collapse to one row per Hugo_Symbol
    file = (file
            .drop(["PATIENT_ID"], axis=1)
            .drop_duplicates(subset=["Hugo_Symbol"], keep="first")
            )

    # reorder columns
    col_reorder = ["Hugo_Symbol","n_patient_mut", "Mut_freq", "avg_OS_months", "Gene_Score",
                   "genomic_alt_cohort", "genomic_alt_rate_cohort" ]
    file = file[col_reorder]

    # write out
    basename = os.path.basename(filepath)
    name, _  = os.path.splitext(basename)
    outpath  = os.path.join(output_dir, f"{name}_scored.csv")
    file.to_csv(outpath, index=False)

In [47]:
# 1. Define input/output directories
input_dir  = "/content/drive/MyDrive/MSK-CHORD-new/cancer_files_scored"
output_dir = "/content/drive/MyDrive/MSK-CHORD-new/cancer_files_scored_filtered"
os.makedirs(output_dir, exist_ok=True)

# Loop through each cancer file
for filepath in glob.glob(os.path.join(input_dir, "*.csv")):
    # read the file
    file_scored = pd.read_csv(filepath)

    # keep teh rows with Mut_freq <0.1
    file_scored = file_scored[file_scored["Mut_freq"] < 0.1]

     # write out
    basename = os.path.basename(filepath)
    name, _  = os.path.splitext(basename)
    outpath  = os.path.join(output_dir, f"{name}_scored_filtered.csv")
    file_scored.to_csv(outpath, index=False)
